In [3]:
import pandas as pd
import numpy as np
import string
import re
from nltk.corpus import stopwords
from autocorrect import spell
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,confusion_matrix,make_scorer,classification_report
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.decomposition import TruncatedSVD

In [4]:
data=pd.read_csv('news_data.csv')

In [5]:
data.rename(columns={'Unnamed: 0':'news_id','text':'news_details'},inplace=True)

In [6]:
data['label']=data['label'].map({'FAKE':0,'REAL':1})

In [7]:
data['news_text']=data['title']+data['news_details']

In [8]:
def convertlower(s):
    return s.lower()
def removeWhiteSpaces(s):
    return s.strip()
def removePunctuations(s):
    return s.translate(str.maketrans('','',string.punctuation))
def removeNumbers(s):
    return re.sub(r'\d+','',s)
def removeURL(s):
    return re.sub(r'^https?:\/\/.*[\r\n]*','',s)
def removeTags(s):
    remove_tags=re.compile('<.*?>')
    return re.sub(remove_tags,'',s)
def removenewline(s):
    return s.replace('\n','') 

In [9]:
    data['news_text']=data['news_text'].apply(convertlower)
    data['news_text']=data['news_text'].apply(removeWhiteSpaces)
    data['news_text']=data['news_text'].apply(removePunctuations)
    data['news_text']=data['news_text'].apply(removeNumbers)
    data['news_text']=data['news_text'].apply(removeURL)
    data['news_text']=data['news_text'].apply(removeTags)
    data['news_text']=data['news_text'].apply(removenewline)

In [10]:
tfidf=TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS,max_df=0.8,min_df=0.2)

In [11]:
tfidf_matrix=tfidf.fit_transform(data['news_text'])

In [12]:
tfidf_matrix.shape

(6335, 88)

In [13]:
X_train,X_test,y_train,y_test=train_test_split(tfidf_matrix,np.array(data['label']),test_size=0.2,random_state=42)

In [14]:
svd=TruncatedSVD(n_components=60)
X_train=svd.fit_transform(X_train)
X_test=svd.transform(X_test)

In [15]:
X_train.shape

(5068, 60)

In [16]:
def svmclf(clf,X,y_true):
    y_pred=clf.predict(X) 
    score=f1_score(y_true,y_pred,average='weighted')
    cm=pd.DataFrame(confusion_matrix(y_true,y_pred),columns=['0','1'],index=['0','1'])
    return score,cm

In [17]:
clf=SVC()
clf.fit(X_train,y_train)

SVC()

In [18]:
score_test,cm_test=svmclf(clf,X_test,y_test)
score_train,cm_train=svmclf(clf,X_train,y_train)

In [19]:
score_test,score_train

(0.8413585258003115, 0.8924489217184637)